In [1]:
import os
from backoff import on_exception, expo
import json
import requests
from collections import Counter
from datetime import datetime
from ratelimit import limits, sleep_and_retry, RateLimitException
import time
import urllib.request
import asyncio
import aiohttp
import nest_asyncio
from async_timeout import timeout
from requests.exceptions import HTTPError

In [2]:
data_path = "D:\\METIS\\data\\"
image_path = "D:\\METIS\\images\\"

images_saved = []
met_objects = []

In [3]:
url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects'
response = requests.get(url, allow_redirects=True)
all_objects = json.loads(response.text)
objectIDs = all_objects['objectIDs']

In [4]:
@sleep_and_retry
@on_exception(expo, RateLimitException, max_tries=8)
@limits(calls=300, period=300)
def get_object_details(object_id, img_url=True):
    url ='https://collectionapi.metmuseum.org/public/collection/v1/objects/{}'.format(object_id)
    response = requests.get(url, timeout=10)
    response_json = json.loads(response.text)
    met_objects.append(response_json)
    
    if img_url:
        return response_json['primaryImage']

@on_exception(expo, RateLimitException, max_tries=8)
@limits(calls=300, period=300)
def get_image(object_id, url):
    filename = image_path + "{}.jpg".format(object_id)
    im = urllib.request.urlretrieve(url, filename)
    images_saved.append({
                        'objectID': object_id,
                        'url': url,
                        'filename': filename
                        })

In [5]:
img_url = get_object_details(1567)
get_image(1567, img_url)

In [6]:
images_saved

[{'objectID': 1567,
  'url': 'https://images.metmuseum.org/CRDImages/ad/original/79684.jpg',
  'filename': 'D:\\METIS\\images\\1567.jpg'}]

In [7]:
images_saved = []
met_objects = []

In [ ]:
start = datetime.now()

for i, object_id in enumerate(objectIDs):
    if i % 1000 == 0:
        with open(data_path + "images_saved.json", "w") as write_file:
            json.dump(images_saved, write_file)   
            
        with open(data_path + "met_objects.json", "w") as write_file:
            json.dump(met_objects, write_file)  
            
        print("{}: {} elapsed".format(i, datetime.now() - start))
            
    try:
        img_url = get_object_details(object_id)
        get_image(object_id, img_url)
        
    except:
        pass
   # print(i, object_id)

0: 0:00:00.009488 elapsed
1000: 0:15:55.030031 elapsed
2000: 0:32:08.139061 elapsed
3000: 0:50:11.821946 elapsed
4000: 1:07:30.602742 elapsed
5000: 1:23:08.001312 elapsed
6000: 1:40:58.846982 elapsed
7000: 1:57:26.045912 elapsed
8000: 2:14:46.693437 elapsed
9000: 2:31:33.445107 elapsed
10000: 2:50:21.025761 elapsed
11000: 3:06:42.417831 elapsed


In [ ]:
with open(data_path + "images_saved.json", "w") as write_file:
    json.dump(images_saved, write_file)   
            
with open(data_path + "met_objects.json", "w") as write_file:
    json.dump(met_objects, write_file)  

In [ ]:
len(images_saved)